In [1]:
import ipyleaflet 
import ipywidgets
import pandas as pd
import time
import os
import asyncio
import logging
from branca.colormap import linear
from random import uniform
from ast import literal_eval

markers = []
ANIMATION_STEP_DELAY = 0.2  # in seconds

m = ipyleaflet.Map(center = (50.0755,14.4), zoom = 10, layout=ipywidgets.Layout(height='720px'))

<H1>Polyline vizualizace</H1>
<H3>Vizualizace cest uživatelů aplikace SmartGuide</H3>
<br>
Vizualizace založeny na datech společnosti SmartGuide
<br>
Vytvořeno v rámci bakalářské práce pro Fakultu Informatiky a Managementu Univerzity Hradce Králové
<br>
<br>
Petr Veselý

In [2]:
data = pd.concat(map(pd.read_csv, ['polylines/polylines_0.csv', 'polylines/polylines_1.csv']), ignore_index=True)
data.points = data.points.apply(literal_eval)
days = sorted(data.date.unique())
dev_languages=sorted(data.dev_language.unique())
dev_os=sorted(data.dev_system.unique())

In [3]:
def update(date=None, lang=None, os=None, tourist=None):          
    if not date and not lang and not os:  
        frame_data = data
        lines.opacity=0.5
    else:
        frame_data = data.loc[data['date'] >= date[0]].loc[data['date'] <= date[1]] 
        lines.opacity=0.75
        if os:
            frame_data = frame_data[frame_data['dev_system'].isin(os)]
        if not tourist:
            if lang:
                frame_data = frame_data[frame_data['dev_language'].isin(lang)]
        else:    
            frame_data = frame_data[frame_data.foreginer.eq(True)]       
     
    lines.locations = frame_data['points'].values.tolist()

In [4]:
slider = ipywidgets.SelectionRangeSlider(description='',
                                         options=days,
                                         readout=True,
                                         continuous_update=False, 
                                         layout=ipywidgets.Layout(width='720px')
)

In [5]:
os_select = ipywidgets.SelectMultiple(
    options= dev_os,
    description='Device OS',
    disabled=False
)

In [6]:
lang_select = ipywidgets.SelectMultiple(
    options= dev_languages,
    description='Language',
    rows=15,
    disabled=False
)

In [7]:
all_langs_button = ipywidgets.Button(
    description='All languages',
    disabled=False,
    tooltip='Select all languages',
    icon='fa-language',
    layout=ipywidgets.Layout(width='150px')
)

def all_langs(_):
    lang_select.value = dev_languages

all_langs_button.on_click(all_langs)

In [8]:
tourist_button = ipywidgets.ToggleButton(
    value=False,
    description='Tourists only',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Show only assumed tourists, only useful within Czechia',
    icon='fa-suitcase'
)

In [9]:
def call_update(change):
   update(slider.value, lang_select.value, os_select.value, tourist_button.value)

slider.observe(call_update, 'value')
os_select.observe(call_update, 'value')
lang_select.observe(call_update, 'value')
tourist_button.observe(call_update, 'value')

In [10]:
def animate_call(_):
    if(animate_button.icon == 'play'):
        animate_button.icon = 'pause'
#         while run_animation_step():
#             time.sleep(0.3)
    else:
        animate_button.icon = 'play'   

In [11]:
def run_animation_step():
    i = 1
    j = 0
   
    while i < len(days) and i != days[i] != slider.value[1] :
        i += 1

    while j < i and j != days[j] != slider.value[0] :
        j += 1
        
    if i + 1 < len(days):
        slider.value = (days[j + 1], days[i+1] )
        return True
    
    animate_button.icon = 'play'
    return False

async def run_animation_loop():
    while True:
        await asyncio.sleep(ANIMATION_STEP_DELAY)
        if animate_button.icon == 'pause':
            run_animation_step()
    return x

In [12]:
def all_data(_):
    print('Prepping full datagram, please wait.')
    update()

In [13]:
animate_button = ipywidgets.Button(
    description='',
    disabled=False,
    tooltip='Animate',
    icon='play', # pause,
    layout=ipywidgets.Layout(width='50px')
)

animate_button.on_click(animate_call)

In [14]:
all_button = ipywidgets.Button(
    description='',
    disabled=False,
    tooltip='All Data!',
    icon='fa-globe',
    layout=ipywidgets.Layout(width='50px')
)

all_button.on_click(all_data)

In [15]:
lines = ipyleaflet.Polyline(
    locations=[],
    color="blue" ,
    fill=False,
    weight=1,
)

m.add_layer(lines)
m.add_control(ipyleaflet.FullScreenControl())

In [16]:
widget_control1 = ipyleaflet.WidgetControl(widget=slider, position='topright', layout=ipywidgets.Layout(width='500px'))
m.add_control(widget_control1)
widget_control2 = ipyleaflet.WidgetControl(widget=animate_button, position='bottomleft')
m.add_control(widget_control2)
widget_control3 = ipyleaflet.WidgetControl(widget=all_button, position='bottomleft')
m.add_control(widget_control3)
widget_control4 = ipyleaflet.WidgetControl(widget=lang_select, position='bottomright')
m.add_control(widget_control4)
widget_control5 = ipyleaflet.WidgetControl(widget=os_select, position='bottomright')
m.add_control(widget_control5)
widget_control6 = ipyleaflet.WidgetControl(widget=tourist_button, position='topright')
m.add_control(widget_control6)
widget_control7 = ipyleaflet.WidgetControl(widget=all_langs_button, position='bottomright')
m.add_control(widget_control7)

In [17]:
display(m)

logging.getLogger('asyncio').setLevel(logging.WARNING)
asyncio.ensure_future(run_animation_loop())
slider.value = (days[0], days[10])
run_animation_step()

Map(center=[50.0755, 14.4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

True

Pro selekci více položek v ui modulech Device OS a Language lze použít klávesy Shift nebo CTRL.